⚪
SQL

# Сервис для чтения книг по подписке

# Задания

Посчитайте, сколько книг вышло после 1 января 2000 года.

Для каждой книги посчитайте количество обзоров и среднюю оценку.

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Как выполнить задание?**

Опишите цели исследования.

Исследуйте таблицы — выведите первые строки.

Сделайте по одному SQL-запросу для решения каждого задания.

Выведите результат каждого запроса в тетрадке.

Опишите выводы по каждой из решённых задач.

# Импорт библиотек и чтение таблиц

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
# Чтение таблиц из базы данных
books = ''' SELECT *
            FROM books
            LIMIT 5;
        '''
pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица **books**  содержит данные о книгах:

book_id — идентификатор книги;

author_id — идентификатор автора;

title — название книги;

num_pages — количество страниц;

publication_date — дата публикации книги;

publisher_id — идентификатор издателя.

In [4]:
authors = ''' SELECT *
            FROM authors
            LIMIT 5;
        '''
pd.io.sql.read_sql(authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица **authors** содержит данные об авторах:

author_id — идентификатор автора;

author — имя автора.

In [5]:
ratings = ''' SELECT *
            FROM ratings
            LIMIT 5;
        '''
pd.io.sql.read_sql(ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица **ratings** содержит данные о пользовательских оценках книг:

rating_id — идентификатор оценки;

book_id — идентификатор книги;

username — имя пользователя, оставившего оценку;

rating — оценка книги.

In [6]:
reviews = ''' SELECT *
            FROM reviews
            LIMIT 5;
        '''
pd.io.sql.read_sql(reviews, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица **reviews** содержит данные о пользовательских обзорах на книги:

review_id — идентификатор обзора;

book_id — идентификатор книги;

username — имя пользователя, написавшего обзор;

text — текст обзора.

# Запросы

## 1. Посчитайте, сколько книг вышло после 1 января 2000 года.

In [7]:
b_2000 = ''' SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01';
'''

pd.io.sql.read_sql(b_2000, con = engine) 

,count
0,819


После 1 января 2000 года выпущено 819 книг.

## 2. Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [8]:
mean_rating = '''SELECT ratings.book_id AS book_id,
        COUNT(reviews.text) AS count_of_reviews,
        AVG(ratings.rating) AS mean_rating
        FROM ratings LEFT JOIN reviews ON (reviews.book_id = ratings.book_id
            AND reviews.username = ratings.username)
        GROUP BY ratings.book_id
        ORDER BY count_of_reviews DESC
        LIMIT 5;    
'''
pd.io.sql.read_sql(mean_rating, con = engine)

,book_id,count_of_reviews,mean_rating
0,948,7,3.662500
1,750,6,4.125000
2,695,6,4.081081
3,963,6,3.977273
4,696,6,3.830508


Максимальное количество обзоров для книги - 7.

<div class="alert alert-warning">
<b>🕵️‍♀️  Комментарий тимлида </b>

Запрос отрабатывает корректно 👍
    
Но для заказчика айди книги может ни о чем не говорить. Было бы удобнее, если бы здесь были названия. 
    
Еще добавлю, что лучше использовать алиасы вместо обращения к таблицам через их полное название. <a href = "https://ru.wikipedia.org/wiki/Alias_(SQL)">Что такое алиас и с чем его едят </a>
    </div>

## 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [9]:
publishing_house = ''' SELECT M.publisher_id, M.publisher, M.count_of_books
    FROM (SELECT books.publisher_id AS publisher_id,
        COUNT(books.book_id) AS count_of_books,
        publishers.publisher AS publisher 
    FROM books LEFT JOIN publishers ON books.publisher_id=publishers.publisher_id
    WHERE num_pages>50
    GROUP BY books.publisher_id, publishers.publisher
    ORDER BY count_of_books DESC) AS M
    LIMIT 10; 
''' 

pd.io.sql.read_sql(publishing_house, con = engine)

,publisher_id,publisher,count_of_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19
5,33,Ballantine Books,19
6,45,Berkley,17
7,46,Berkley Books,14
8,284,St. Martin's Press,14
9,83,Delta,13


Издательство, выпустившее наибольшее число книг толще 50 страниц - Penguin Books

## 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [10]:
best_author = '''SELECT author.author,
    AVG(avg_rating) as mean_rating
    FROM books book INNER JOIN 
    (SELECT book_id,
        COUNT(rating_id) as ratings_count,
        AVG(rating) as avg_rating
    FROM ratings
    GROUP BY book_id 
    HAVING COUNT(rating_id) >= 50) AS b ON b.book_id = book.book_id 
    INNER JOIN authors author ON author.author_id = book.author_id
    GROUP BY author.author_id, author.author
    ORDER BY mean_rating DESC
    LIMIT 1;
'''
pd.io.sql.read_sql(best_author , con = engine)


,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.283844


## 5.Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
users_reviews = '''SELECT ROUND(AVG(n.count))
FROM (SELECT sum.username, COUNT(sum.username)
    FROM 
    (SELECT username, 
    COUNT(rating_id) AS count
    FROM ratings 
    GROUP BY username) as sum 
    JOIN reviews ON reviews.username = sum.username
    WHERE sum.count > 50
    GROUP BY sum.username) as n;
            
'''

pd.io.sql.read_sql(users_reviews, con = engine)

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.

# Итог.

После 1 января 2000 года было выпущено 819 книг.

Наибольшее число обзоров - 7.

Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books.

Автор с самой высокой средней оценкой книг (только книги с 50 и более оценками) - J.K. Rowling/Mary GrandPré. Рейтинг 4.28.

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.